In [ ]:
# 영상 출력 없이 결과값만 저장 / 3초에 1번씩 프레임 처리

import os
import glob
import json
import tensorflow as tf
import numpy as np
import cv2
import mediapipe as mp
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

emotions = {
    0: ['Angry', (0, 0, 255), (255, 255, 255)], 
    1: ['Disgust', (0, 102, 0), (255, 255, 255)],
    2: ['Fear', (255, 255, 153), (0, 51, 51)],
    3: ['Happy', (153, 0, 153), (255, 255, 255)],
    4: ['Sad', (255, 0, 0), (255, 255, 255)],
    5: ['Surprise', (0, 255, 0), (255, 255, 255)],
    6: ['Neutral', (160, 160, 160), (255, 255, 255)]
}                       # 감정 값
num_classes = len(emotions)
input_shape = (48, 48, 1) # 감정 분석 위치 좌표
weights_1 = 'vggnet.h5'
weights_2 = 'vggnet_up.h5'


class VGGNet(Sequential):
    def __init__(self, input_shape, num_classes, checkpoint_path, lr=1e-3):
        super().__init__()
        self.add(Rescaling(1. / 255, input_shape=input_shape))
        self.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'))
        self.add(BatchNormalization())
        self.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.5))

        self.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.4))

        self.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.5))

        self.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D())
        self.add(Dropout(0.4))

        self.add(Flatten())

        self.add(Dense(1024, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(256, activation='relu'))

        self.add(Dense(num_classes, activation='softmax'))

        self.compile(optimizer=Adam(learning_rate=lr),
                     loss=categorical_crossentropy,
                     metrics=['accuracy'])

        self.checkpoint_path = checkpoint_path


model_1 = VGGNet(input_shape, num_classes, weights_1)
model_1.load_weights(model_1.checkpoint_path)

model_2 = VGGNet(input_shape, num_classes, weights_2)
model_2.load_weights(model_2.checkpoint_path)

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)


def detection_preprocessing(image, h_max=360):
    h, w, _ = image.shape
    if h > h_max:
        ratio = h_max / h
        w_ = int(w * ratio)
        image = cv2.resize(image, (w_, h_max))
    return image


def resize_face(face):
    x = tf.expand_dims(tf.convert_to_tensor(face), axis=2)
    return tf.image.resize(x, (48, 48))


def recognition_preprocessing(faces):
    x = tf.convert_to_tensor([resize_face(f) for f in faces])
    return x


def inference(image):

    global emotion_counts
    H, W, _ = image.shape

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_image)


    if results.detections:
        for detection in results.detections:
            box = detection.location_data.relative_bounding_box
            x = int(box.xmin * W)
            y = int(box.ymin * H)
            w = int(box.width * W)
            h = int(box.height * H)

            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(x + w, W)
            y2 = min(y + h, H)

            face = image[y1:y2, x1:x2]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            face = tf.expand_dims(tf.convert_to_tensor(face), axis=2)
            face = tf.image.resize(face, (48, 48))

            # 밖에서 같은 행동을 또 하고 있기 때문에 안쪽 for문에 넣어서 한번에 처리
            x = recognition_preprocessing(face)
            y_1 = model_1.predict(x)
            y_2 = model_2.predict(x)
            l = np.argmax(y_1 + y_2, axis=1)
            emotion_counts[l[0]] += 1


# Video 감정분석
def save_emotion_results(video_path, emotion_counts):
    # 각 동영상에 대한 감정 분석 결과를 저장할 딕셔너리 생성
    emotion_results = {
        "emotion_counts": emotion_counts,
        "emotion_names": [emotions[label][0] for label in emotion_counts.keys()],
        "emotion_values": list(emotion_counts.values())
    }

    # 개별 동영상 결과를 저장
    json_file_path = f'emotion_results_{os.path.basename(video_path)}.json'
    json_file_path = os.path.join(video_folder_path, json_file_path)

    with open(json_file_path, 'w') as json_file:
        json.dump(emotion_results, json_file, indent=4)

    return emotion_results


# 모든 동영상 결과를 저장할 빈 딕셔너리 초기화
combined_results = {}

# 동영상 파일이 있는 폴더 경로
video_folder_path = 'C:/Users/user/Downloads/test_media'

# 폴더 내의 모든 동영상 파일 경로를 얻어옴
video_paths = glob.glob(os.path.join(video_folder_path, '*.mp4'))

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)

    # Load the video duration for accurate interval processing
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    video_duration = total_frames / fps

    # 감정별 카운트 초기화 (각 프레임당 한 번 초기화)
    emotion_counts = {emotion_label: 0 for emotion_label in emotions}

    # 3초 카운터
    idx = 0

    start_time = time.time()
    
    while True:
        success, image = cap.read()
        if not success:
            break

        current_time = time.time()
        elapsed_time = current_time - start_time

        # 3초당 1번씩 처리 (간격을 더 짧게 조정하여 자주 감정 분석)
        # 초 처리
        seconds = np.floor(cap.get(cv2.CAP_PROP_POS_MSEC)/ 1000) * 1000 - idx
        if seconds % 3 == 0:
            idx += 3
            # 'inference' 이미지 가정 추정
            inference(image)  # 'inference' 함수에서 얼굴을 가져옵니다

            # 초기화 및 최종 결과 저장
            # 결과 저장하는 시간이 처리하는 시간이랑 똑같다.
            # if elapsed_time >= 3.0:
            save_emotion_results(video_path, emotion_counts)
            start_time = time.time()

    cap.release()

    # 통합된 딕셔너리에 결과 저장
    combined_results[os.path.basename(video_path)] = emotion_counts

# 통합된 결과를 하나의 JSON 파일로 저장
combined_json_file_path = os.path.join(video_folder_path, 'combined_emotion_results.json')
with open(combined_json_file_path, 'w') as json_file:
    json.dump(combined_results, json_file, indent=4)

print(f"통합된 감정 분석 결과가 {combined_json_file_path}에 저장되었습니다.")

In [1]:
import os
import json
import tensorflow as tf
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array

# 감정 레이블 정의
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7개의 감정 클래스에 대한 softmax 출력
])

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 학습된 모델 가중치 불러오기 (필요시)
# model.load_weights('your_model_weights.h5')

# mediapipe 모듈 초기화
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# 감정 카운트 초기화
emotion_counts = {label: 0 for label in emotion_labels}

# 동영상 파일 경로
video_path = '../강의'

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR을 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 얼굴 검출
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            # 얼굴 영역 추출 및 전처리
            face = cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)
            face = cv2.resize(face, (48, 48))
            face = img_to_array(face)
            face = np.expand_dims(face, axis=0)
            face = face / 255.0  # 정규화

            # 감정 분석
            emotion_probabilities = model.predict(face)
            emotion_label = emotion_labels[np.argmax(emotion_probabilities)]
            emotion_counts[emotion_label] += 1

            # 결과를 프레임에 표시
            cv2.putText(frame, f'Emotion: {emotion_label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('Emotion Analysis', frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 결과 출력
print("Emotion Counts:", emotion_counts)

# 결과 저장 (JSON 파일로)
result_data = {
    'emotion_counts': emotion_counts,
    'emotion_labels': emotion_labels
}

with open('emotion_result.json', 'w') as json_file:
    json.dump(result_data, json_file, indent=4)


Emotion Counts: {'Angry': 0, 'Disgust': 0, 'Fear': 0, 'Happy': 0, 'Sad': 0, 'Surprise': 0, 'Neutral': 0}


In [35]:
# 되긴하는데 별로임 얼굴인식이 계속도ㅐ서 속도가 1.5배 걸림
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# 얼굴 감지 모델 초기화
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# 감정 분석 모델 초기화 (간단한 CNN 모델 예제)
emotion_model = load_model('emotion_model.h5')  # 감정 분석 모델로 실제 사용되는 모델로 변경해야 함

# 감정 레이블
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 비디오 캡처 초기화
cap = cv2.VideoCapture('../연습1.mp4')  # 감정 분석을 수행할 동영상 파일로 변경

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 얼굴 감지
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            # 얼굴 이미지 추출
            face_image = frame[y:y+h, x:x+w]

            # 감정 분석을 위한 전처리
            face_image = cv2.cvtColor(cv2.resize(face_image, (48, 48)), cv2.COLOR_BGR2GRAY)
            face_image = np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])

            # 감정 분석 모델에 전달
            emotion_prediction = emotion_model.predict(face_image)
            emotion_label = np.argmax(emotion_prediction)

            # 결과 시각화
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, emotions[emotion_label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # 결과 동영상으로 출력
    cv2.imshow('Emotion Analysis', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 11ms/step


In [37]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('emotion_model.h5')
# emotion_model = load_model('vggnet.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일 경로
video_path = '../연습.mp4'

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

while True:
    # 동영상에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break
    
    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # 각 얼굴에 대해 감정 분석 수행
    for (x, y, w, h) in faces:
        face = gray[y:y + h, x:x + w]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0  # 이미지 정규화

        # 모델 입력 형식으로 변환
        face = tf.expand_dims(face, axis=-1)
        face = tf.expand_dims(face, axis=0)

        # 감정 분석 수행
        emotion_prediction = emotion_model.predict(face)
        emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

        # 감정 레이블 표시
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)
        
        # 얼굴 주변에 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 동영상에 프레임 표시
    cv2.imshow('Emotion Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 75ms/step


In [38]:
# 얼굴이 하나만 인지되도록 바꿈 
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('emotion_model.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일 경로
video_path = '../연습.mp4'

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

while True:
    # 동영상에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break
    
    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # 얼굴이 하나 이상 감지되었을 때만 처리
    if len(faces) > 0:
        # 가장 첫 번째 얼굴에 대해서만 처리
        x, y, w, h = faces[0]
        face = gray[y:y + h, x:x + w]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0  # 이미지 정규화

        # 모델 입력 형식으로 변환
        face = tf.expand_dims(face, axis=-1)
        face = tf.expand_dims(face, axis=0)

        # 감정 분석 수행
        emotion_prediction = emotion_model.predict(face)
        emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

        # 감정 레이블 표시
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)
        
        # 얼굴 주변에 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 동영상에 프레임 표시
    cv2.imshow('Emotion Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 40ms/step


In [9]:
# 5프레임마다 감정분석 진행되도록 바꿈
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('emotion_model.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일 경로
video_path = '../연습.mp4'

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

frame_count = 0  # 현재 프레임 카운트 초기화

while True:
    # 동영상에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 얼굴이 하나 이상 감지되었을 때만 처리
    if len(faces) > 0:
        # 가장 첫 번째 얼굴에 대해서만 처리
        x, y, w, h = faces[0]
        face = gray[y:y + h, x:x + w]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0  # 이미지 정규화

        # 5프레임마다 감정 분석 수행
        if frame_count % 5 == 0:
            # 모델 입력 형식으로 변환
            face = tf.expand_dims(face, axis=-1)
            face = tf.expand_dims(face, axis=0)

            # 감정 분석 수행
            emotion_prediction = emotion_model.predict(face)
            emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

            # 감정 레이블 표시
            cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)

        # 얼굴 주변에 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 동영상 표시
    cv2.imshow('Emotion Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1  # 프레임 카운트 증가

# 리소스 해제
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 16ms/step


In [8]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import json

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('emotion_model.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일 경로
video_path = '../연습.mp4'

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# 감정별 횟수 카운트
emotion_counts = {emotion: 0 for emotion in emotion_labels.values()}

# 감정 분석 수행 간격
interval = 5
frame_count = 0

while True:
    # 동영상에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 얼굴이 하나 이상 감지되었을 때만 처리
    if len(faces) > 0:
        # 가장 첫 번째 얼굴에 대해서만 처리
        x, y, w, h = faces[0]
        face = gray[y:y + h, x:x + w]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0  # 이미지 정규화

        # 모델 입력 형식으로 변환
        face = tf.expand_dims(face, axis=-1)
        face = tf.expand_dims(face, axis=0)

        # 감정 분석 수행
        emotion_prediction = emotion_model.predict(face)
        emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

        # 감정 레이블 표시
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)

        # 얼굴 주변에 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # 감정 카운트 증가
        emotion_counts[emotion_label] += 1

    # 동영상에 프레임 표시
    cv2.imshow('Emotion Detection', frame)

    # 5프레임마다 감정 분석
    if frame_count % interval == 0:
        # json 파일로 저장
        with open('emotion_counts.json', 'w') as f:
            json.dump(emotion_counts, f, indent=4)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1  # 프레임 카운트 증가

# 리소스 해제
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 19ms/step


In [2]:
# 5프레임마다 감정분석 진행되고 json 파일에 남는다.
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
from collections import OrderedDict

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('../emotion_model.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일 경로
video_path = '../연습.mp4'

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# 감정별 횟수 카운트
emotion_counts = {emotion: 0 for emotion in emotion_labels.values()}

# 감정 수행 간격을 확인하기 위한 카운트
frame_count = 0  # 현재 프레임 카운트 초기화

while True:
    # 동영상에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=8, minSize=(30, 30))

    # 얼굴이 하나 이상 감지되었을 때만 처리
    if len(faces) > 0:
        # 가장 첫 번째 얼굴에 대해서만 처리
        x, y, w, h = faces[0]
        face = gray[y:y + h, x:x + w]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0  # 이미지 정규화

        # 5프레임마다 감정 분석 수행
        if frame_count % 5 == 0:
            # 모델 입력 형식으로 변환
            face = tf.expand_dims(face, axis=-1)
            face = tf.expand_dims(face, axis=0)

            # 감정 분석 수행
            emotion_prediction = emotion_model.predict(face)
            emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

            # 감정 레이블 표시
            cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)

            # 감정 카운트 증가
            emotion_counts[emotion_label] += 1

        # 얼굴 주변에 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 동영상 표시
    cv2.imshow('Emotion Detection', frame) 

    if frame_count % 5 == 0:
        # json 파일로 저장
        with open('emotion_counts.json', 'w') as f:
            json.dump(emotion_counts, f, indent=4)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1  # 프레임 카운트 증가

# 리소스 해제
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step


In [1]:
# 동영상 여러개 넣으면 데이터가 쌓이도록 만듬 동영상이 한개만 됨 ㅋㅋ
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
from collections import OrderedDict

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('../emotion_model.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일 경로
video_path = '../연습강의/01강유리수의소수표현(1)_[발전]EBS중학뉴런수학2(상).mp4'

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# 감정별 횟수 카운트
emotion_counts = {emotion: 0 for emotion in emotion_labels.values()}

# 감정 수행 간격을 확인하기 위한 카운트
frame_count = 0  # 현재 프레임 카운트 초기화

while True:
    # 동영상에서 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 감지
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=8, minSize=(30, 30))

    # 얼굴이 하나 이상 감지되었을 때만 처리
    if len(faces) > 0:
        # 가장 첫 번째 얼굴에 대해서만 처리
        x, y, w, h = faces[0]
        face = gray[y:y + h, x:x + w]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0  # 이미지 정규화

        # 5프레임마다 감정 분석 수행
        if frame_count % 5 == 0:
            # 모델 입력 형식으로 변환
            face = tf.expand_dims(face, axis=-1)
            face = tf.expand_dims(face, axis=0)

            # 감정 분석 수행
            emotion_prediction = emotion_model.predict(face)
            emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

            # 감정 레이블 표시
            cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)

            # 감정 카운트 증가
            emotion_counts[emotion_label] += 1

        # 얼굴 주변에 사각형 그리기
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 동영상 표시
    cv2.imshow('Emotion Detection', frame) 

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1  # 프레임 카운트 증가

# 리소스 해제
cap.release()
cv2.destroyAllWindows()

plate_data = OrderedDict()

try:
    with open('data_Emotional_ex.json','r')as f:
        plate_data = json.load(f)
except FileNotFoundError:
    plate_data = {'data' : []}

plate_data['data'].append(emotion_counts)
with open('./data_Emotional_ex.json','w') as f:
    json.dump(plate_data, f, ensure_ascii=False, indent=4)


OSError: No file or directory found at ../emotion_model.h5

In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
import os
from collections import OrderedDict

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('../../emotion_model.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일이 있는 폴더 경로
# folder_path = 'D:\ebs\변창현'  
folder_path = 'D:\ebs\EBS중학뉴런수학2(상)_손석민 - 복사본' 

# 결과를 저장할 JSON 파일 경로
output_json_path = 'data_Emotional.json'

# 결과를 저장할 OrderedDict 초기화
emotion_data = OrderedDict()
emotion_data['data'] = []

# 폴더 안에 있는 동영상 파일들을 대상으로 처리
for file_name in os.listdir(folder_path):
    if file_name.endswith('.mp4'):
        video_path = os.path.join(folder_path, file_name)

        # 동영상 파일 열기
        cap = cv2.VideoCapture(video_path)

        # 감정별 횟수 카운트 초기화
        emotion_counts = {emotion: 0 for emotion in emotion_labels.values()}

        # 감정 수행 간격을 확인하기 위한 카운트 초기화
        frame_count = 0  

        while True:
            # 동영상에서 프레임 읽기
            ret, frame = cap.read()
            if not ret:
                break

            # 그레이스케일로 변환
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # 얼굴 감지
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=8, minSize=(30, 30))

            # 얼굴이 하나 이상 감지되었을 때만 처리
            if len(faces) > 0:
                # 가장 첫 번째 얼굴에 대해서만 처리
                x, y, w, h = faces[0]
                face = gray[y:y + h, x:x + w]
                face = cv2.resize(face, (48, 48))
                face = face / 255.0  # 이미지 정규화

                # 5프레임마다 감정 분석 수행
                if frame_count % 5 == 0:
                    # 모델 입력 형식으로 변환
                    face = tf.expand_dims(face, axis=-1)
                    face = tf.expand_dims(face, axis=0)

                    # 감정 분석 수행
                    emotion_prediction = emotion_model.predict(face)
                    emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

                    # 감정 카운트 증가
                    emotion_counts[emotion_label] += 1

            frame_count += 1  # 프레임 카운트 증가

        # 리소스 해제
        cap.release()

        # 결과를 OrderedDict에 추가
        emotion_data['data'].append({video_path: emotion_counts})

        # OrderedDict를 JSON 파일로 저장
        with open(output_json_path, 'w') as f:
            json.dump(emotion_data, f, ensure_ascii=False, indent=4)

1/1 [==============================] - 0s 13ms/step


In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
import os
from collections import OrderedDict

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('../../emotion_model.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일이 있는 폴더 경로
# folder_path = 'D:\ebs\변창현'  
folder_path = 'D:\ebs\[발전]EBS중학뉴런수학2(상)_변창현' 

# 결과를 저장할 JSON 파일 경로
output_json_path = 'data_Emotional_1.json'

# 결과를 저장할 OrderedDict 초기화
emotion_data = OrderedDict()
emotion_data['data'] = []

# 폴더 안에 있는 동영상 파일들을 대상으로 처리
for file_name in os.listdir(folder_path):
    if file_name.endswith('.mp4'):
        video_path = os.path.join(folder_path, file_name)

        # 동영상 파일 열기
        cap = cv2.VideoCapture(video_path)

        # 감정별 횟수 카운트 초기화
        emotion_counts = {emotion: 0 for emotion in emotion_labels.values()}

        # 감정 수행 간격을 확인하기 위한 카운트 초기화
        frame_count = 0  

        while True:
            # 동영상에서 프레임 읽기
            ret, frame = cap.read()
            if not ret:
                break

            # 그레이스케일로 변환
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # 얼굴 감지
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=8, minSize=(30, 30))

            # 얼굴이 하나 이상 감지되었을 때만 처리
            if len(faces) > 0:
                # 가장 첫 번째 얼굴에 대해서만 처리
                x, y, w, h = faces[0]
                face = gray[y:y + h, x:x + w]
                face = cv2.resize(face, (48, 48))
                face = face / 255.0  # 이미지 정규화

                # 5프레임마다 감정 분석 수행
                if frame_count % 5 == 0:
                    # 모델 입력 형식으로 변환
                    face = tf.expand_dims(face, axis=-1)
                    face = tf.expand_dims(face, axis=0)

                    # 감정 분석 수행
                    emotion_prediction = emotion_model.predict(face)
                    emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

                    # 감정 카운트 증가
                    emotion_counts[emotion_label] += 1

            frame_count += 1  # 프레임 카운트 증가

        # 리소스 해제
        cap.release()

        # 결과를 OrderedDict에 추가
        emotion_data['data'].append({video_path: emotion_counts})

        # OrderedDict를 JSON 파일로 저장
        with open(output_json_path, 'w') as f:
            json.dump(emotion_data, f, ensure_ascii=False, indent=4)

In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
import os
from collections import OrderedDict

# 얼굴 감지기 초기화 (Haar Cascade 사용)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 분석 모델 로드
emotion_model = load_model('../../emotion_model.h5')

# 감정 레이블
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# 동영상 파일이 있는 폴더 경로
# folder_path = 'D:\ebs\변창현'  
folder_path = 'D:\ebs\[발전]EBS중학뉴런수학2(상)_변창현 - 복사본' 

# 결과를 저장할 JSON 파일 경로
output_json_path = 'data_Emotional_1_1.json'

# 결과를 저장할 OrderedDict 초기화
emotion_data = OrderedDict()
emotion_data['data'] = []

# 폴더 안에 있는 동영상 파일들을 대상으로 처리
for file_name in os.listdir(folder_path):
    if file_name.endswith('.mp4'):
        video_path = os.path.join(folder_path, file_name)

        # 동영상 파일 열기
        cap = cv2.VideoCapture(video_path)

        # 감정별 횟수 카운트 초기화
        emotion_counts = {emotion: 0 for emotion in emotion_labels.values()}

        # 감정 수행 간격을 확인하기 위한 카운트 초기화
        frame_count = 0  

        while True:
            # 동영상에서 프레임 읽기
            ret, frame = cap.read()
            if not ret:
                break

            # 그레이스케일로 변환
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # 얼굴 감지
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=8, minSize=(30, 30))

            # 얼굴이 하나 이상 감지되었을 때만 처리
            if len(faces) > 0:
                # 가장 첫 번째 얼굴에 대해서만 처리
                x, y, w, h = faces[0]
                face = gray[y:y + h, x:x + w]
                face = cv2.resize(face, (48, 48))
                face = face / 255.0  # 이미지 정규화

                # 5프레임마다 감정 분석 수행
                if frame_count % 5 == 0:
                    # 모델 입력 형식으로 변환
                    face = tf.expand_dims(face, axis=-1)
                    face = tf.expand_dims(face, axis=0)

                    # 감정 분석 수행
                    emotion_prediction = emotion_model.predict(face)
                    emotion_label = emotion_labels[tf.argmax(emotion_prediction, axis=1).numpy()[0]]

                    # 감정 카운트 증가
                    emotion_counts[emotion_label] += 1

            frame_count += 1  # 프레임 카운트 증가

        # 리소스 해제
        cap.release()

        # 결과를 OrderedDict에 추가
        emotion_data['data'].append({video_path: emotion_counts})

        # OrderedDict를 JSON 파일로 저장
        with open(output_json_path, 'w') as f:
            json.dump(emotion_data, f, ensure_ascii=False, indent=4)

1/1 [==============================] - 0s 38ms/step
